In [1]:
import torch
from torch.utils import data
import numpy
import pandas
import cv2
import torchvision.transforms as transforms
from datasets import datasets
import torch.nn as nn
torch.set_default_tensor_type(torch.FloatTensor)

In [6]:
datas=datasets.Pnet_dataset()
loader=data.DataLoader(datas,batch_size=1,shuffle=False,num_workers=1)

In [7]:
from model.PNet import PNet
net=PNet()
device=torch.device('cuda:0')
net.load_state_dict(torch.load('./pretrained_model/new_PNet6.pth'))
net=net.to(device)

In [5]:
for m in net.modules():
    if isinstance(m,nn.Conv2d):
        nn.init.normal_(m.weight)
        nn.init.xavier_normal_(m.weight)
        nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.bias,0)
    elif isinstance(m,nn.Linear):
        nn.init.normal_(m.weight)

In [10]:
import torch.optim as optim
criterion1=nn.CrossEntropyLoss()
criterion2=nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=0.000001,betas=(0.9,0.99))

In [14]:
beta1=[1,1,0]
beta2=[0,1,1]
for epoch in range(2):
    running_loss=0.0
    for i,data in enumerate(loader):
        img,det,label= data
        img=img.type(torch.DoubleTensor)
        img,det,label = img.to(device),det.to(device),label.to(device)
        optimizer.zero_grad()
        outputs1,outputs2,_=net(img)
        outputs1.squeeze_(3),outputs2.squeeze_(3)
        outputs1.squeeze_(2),outputs2.squeeze_(2)
        det.squeeze_(1)
        if det==2:
            loss=criterion2(outputs2,label)*0.5
        elif det==0:
            loss=criterion1(outputs1,det)
        elif det==1:
            loss=criterion1(outputs1,det)+criterion2(outputs2,label)*0.5
        if loss.item()>5:
            continue
        loss=loss.type(torch.DoubleTensor)
        #loss=torch.DoubleTensor(loss)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%2000==1999:
            print('[%d,%5d] loss:%.3f' %(epoch+1,i+1,running_loss/2000))
            running_loss=0.0
print('training finished')

[1, 2000] loss:0.025
[1, 4000] loss:0.036
[1, 6000] loss:0.028
[1, 8000] loss:0.029
[1,10000] loss:0.042
[1,12000] loss:0.028
[1,14000] loss:0.025
[1,16000] loss:0.032
[1,18000] loss:0.019
[1,20000] loss:0.026
[1,22000] loss:0.024
[1,24000] loss:0.033
[1,26000] loss:0.024
[1,28000] loss:0.026
[1,30000] loss:0.026
[1,32000] loss:0.031
[1,34000] loss:0.025
[1,36000] loss:0.026
[1,38000] loss:0.029
[1,40000] loss:0.025
[1,42000] loss:0.023
[1,44000] loss:0.036
[1,46000] loss:0.031
[1,48000] loss:0.026
[1,50000] loss:0.023
[1,52000] loss:0.024
[1,54000] loss:0.043
[1,56000] loss:0.039
[1,58000] loss:0.025
[1,60000] loss:0.022
[1,62000] loss:0.025
[1,64000] loss:0.021
[1,66000] loss:0.037
[1,68000] loss:0.034
[1,70000] loss:0.020
[1,72000] loss:0.029
[1,74000] loss:0.027
[1,76000] loss:0.028
[1,78000] loss:0.031
[1,80000] loss:0.016
[1,82000] loss:0.028
[1,84000] loss:0.033
[1,86000] loss:0.026
[1,88000] loss:0.034
[1,90000] loss:0.039
[1,92000] loss:0.022
[1,94000] loss:0.035
[1,96000] los

KeyboardInterrupt: 

In [15]:
torch.save(net.state_dict(),'./pretrained_model/new_PNet7.pth')

In [15]:
loss2

tensor(4.9724, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)

In [8]:
loss.type(torch.double)

tensor(4.9724, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)

In [18]:
a=torch.tensor([2.,3.])

In [20]:
a.dtype

torch.float32

In [21]:
a.type(torch.DoubleTensor)

tensor([2., 3.], dtype=torch.float64)

In [12]:
stride=2
scale=1
k=1
cellsize=12
offset=numpy.array([
        [[stride*i/scale**k for i in range(5)] for _ in range(6)],
        [[stride*j/scale**k for _ in range(5)] for j in range(6)],
        [[(stride*i+cellsize)/scale**k for i in range(5)] for _ in range(6)],
        [[(stride*j+cellsize)/scale**k for _ in range(5)] for j in range(6)]
        ])

In [15]:
offset*2/5

array([[[0. , 0.8, 1.6, 2.4, 3.2],
        [0. , 0.8, 1.6, 2.4, 3.2],
        [0. , 0.8, 1.6, 2.4, 3.2],
        [0. , 0.8, 1.6, 2.4, 3.2],
        [0. , 0.8, 1.6, 2.4, 3.2],
        [0. , 0.8, 1.6, 2.4, 3.2]],

       [[0. , 0. , 0. , 0. , 0. ],
        [0.8, 0.8, 0.8, 0.8, 0.8],
        [1.6, 1.6, 1.6, 1.6, 1.6],
        [2.4, 2.4, 2.4, 2.4, 2.4],
        [3.2, 3.2, 3.2, 3.2, 3.2],
        [4. , 4. , 4. , 4. , 4. ]],

       [[4.8, 5.6, 6.4, 7.2, 8. ],
        [4.8, 5.6, 6.4, 7.2, 8. ],
        [4.8, 5.6, 6.4, 7.2, 8. ],
        [4.8, 5.6, 6.4, 7.2, 8. ],
        [4.8, 5.6, 6.4, 7.2, 8. ],
        [4.8, 5.6, 6.4, 7.2, 8. ]],

       [[4.8, 4.8, 4.8, 4.8, 4.8],
        [5.6, 5.6, 5.6, 5.6, 5.6],
        [6.4, 6.4, 6.4, 6.4, 6.4],
        [7.2, 7.2, 7.2, 7.2, 7.2],
        [8. , 8. , 8. , 8. , 8. ],
        [8.8, 8.8, 8.8, 8.8, 8.8]]])

In [18]:
numpy.array([3,4,3,4])*numpy.array([0.2,0.3,0.7,0.2])+[1,2,3,4]

array([1.6, 3.2, 5.1, 4.8])